In [ ]:
!pip install lime
from lime.lime_text import LimeTextExplainer

In [ ]:
def check_inference(model, processor, prompts, max_new_tokens=50):
    tokenizer = processor.tokenizer
    bad_words = ["<image>", "<fake_token_around_image>"]
    if len(bad_words) > 0:
        bad_words_ids = tokenizer(bad_words, add_special_tokens=False).input_ids

    eos_token = "</s>"
    eos_token_id = tokenizer.convert_tokens_to_ids(eos_token)

    inputs = processor(prompts, return_tensors="pt").to(device)
    generated_ids = model.generate(**inputs, eos_token_id=[eos_token_id], bad_words_ids=bad_words_ids, max_new_tokens=max_new_tokens, early_stopping=False)
    generated_text = processor.batch_decode(generated_ids, skip_special_tokens=True)[0]
    return generated_text

In [ ]:
def extract_probabilities(generated_text):
    emotion = extract_second_assistant_emotion(generated_text)
    emotions = ["Amusement", "Awe", "Contentment", "Excitement", "Fear", "Sadness", "Anger", "Disgust"]
    probabilities = np.zeros(len(emotions))
    if emotion in emotions:
        index = emotions.index(emotion)
        probabilities[index] = 1.0  
    return probabilities


In [ ]:
def predict_proba(texts):
    results = []
    for text in texts:
        url = 'https://via.placeholder.com/100'
        prompts = [
    [
        "User: Wonderful detail, the sky contrasts nicely with the rest of the painting. The defining outlines of the mother and child are very effective in evoking emotions. Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
        url,
        "<end_of_utterance>",
        "\nAssistant: Awe.<end_of_utterance>",
        "\nUser:",
        url,
        f"{texts} Choose one emotion from: Amusement, Awe, Contentment, Excitement, Fear, Sadness, Anger.",
        "\nAssistant:",
    ],
]
        generated_text = check_inference(model, processor, prompts)

        emotion_probabilities = extract_probabilities(generated_text)
        results.append(emotion_probabilities)
    return np.array(results)


In [ ]:
class_names = ["Amusement", "Awe", "Contentment", "Excitement", "Fear", "Sadness", "Anger", "Disgust"]
explainer = LimeTextExplainer(class_names=class_names)

In [ ]:
data = {'utterance': ["This abstract shows how small the world is. The landscape looks like a gorgeous winter evening, with grounds covered in snow. I can imagine the sounds of whistling winds and howling wolves."]}
sample_df = pd.DataFrame(data)

text_instance = sample_df.iloc[0]['utterance']

probabilities = predict_proba([text_instance])

print("Predicted probabilities:", probabilities)

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:563: UserWarning: `num_beams` is set to 1. However, `early_stopping` is set to `True` -- this flag is only used in beam-based generation modes. You should set `num_beams>1` or unset `early_stopping`.
  warnings.warn(


Predicted probabilities: [[0. 1. 0. 0. 0. 0. 0. 0.]]


In [ ]:
torch.cuda.empty_cache()
exp = explainer.explain_instance(text_instance, predict_proba, num_features=6)

print(exp.as_list())
exp.show_in_notebook(text=True)

Token indices sequence length is longer than the specified maximum sequence length for this model (155478 > 2048). Running this sequence through the model will result in indexing errors


In [ ]:
def extract_assistant_emotion(text):
    # Find all occurrences of "Assistant: " in the text
    parts = re.split(r'Assistant: ', text)

    # If there are at least two occurrences
    if len(parts) > 1:
        # Extract the part after the second "Assistant: "
        last_assistant_part = parts[-1]

        # Find the first emotion word from the specified list
        emotions = ['Amusement', 'Awe', 'Contentment', 'Excitement', 'Fear', 'Sadness', 'Anger', 'Disgust']
        for emotion in emotions:
            if emotion in last_assistant_part:
                return emotion

    # Return None if no emotion is found
    return None